In [2]:
!pip install imageio
!pip install opencv-python
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-52oul_45
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-52oul_45
     |████████████████████████████████| 1.0MB 12.5MB/s 
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0a8a2778e805509eb0ec0c2487732a9085db33900_-cp36-none-any.whl size=146834 sha256=2bba89bb50968e1a12f3d23c0007b564dcda47989f43231717176cfb7170b40b
  Stored in directory: /tmp/pip-ephem-wheel-cache-z9v6vgbt/wheels/eb/1b/35/fce87697be00d2fc63e0b4b395b0d9c7e391a10e98d9a0d97f
Successfully built tensorflow-docs
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [4]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

import random
import re
import os
import tempfile
import numpy as np
import ssl 
import cv2
import imageio
from IPython import display
import requests
from urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

logging.set_verbosity(logging.ERROR)
_CACHE_DIR = tempfile.mkdtemp()

In [6]:
# get Kinetics 400 labels
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
labels = requests.get(KINETICS_URL).content.splitlines()
labels = [x.decode("UTF-8") for x in labels]
print("Found %d labels." % len(labels))

Found 400 labels.


In [22]:
# get list of videos in UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
unverified_context = ssl._create_unverified_context()

htmlpage = requests.get(UCF_ROOT, verify=False).content.decode("utf-8")
video_links_list = re.findall("(v_[\w_]+\.avi)", htmlpage)
ucf101_video_list = sorted(set(video_links_list))
print("Found %d video names in UCF101 dataset." % len(ucf101_video_list))

categories = {}
for video_name in ucf101_video_list:
  category = video_name[2:-12]
  if category not in categories:
    categories[category] = 1
  else:
    categories[category] += 1
print("Found %d categories in UCF101 dataset." % len(categories))

Found 13320 video names in UCF101 dataset.
Found 101 categories in UCF101 dataset.


In [ ]:
for category, size in categories.items():
  print("%-20s %4d videos" % (category, size))

In [34]:
# download video
video_name = "v_Biking_g01_c01.avi"
video_path = os.path.join(_CACHE_DIR, video_name)
if not os.path.exists(video_path):
  video_link_url = UCF_ROOT + '/' + video_name
  binary_data = requests.get(video_link_url, verify=False).content
  open(video_path, "wb").write(binary_data)

# load video frames
target_size = (224, 224)
frames = []
cap = cv2.VideoCapture(video_path)
try:
  while True:
    ret, frame = cap.read()
    if not ret:
      break
    # crop center square
    h, w, _ = frame.shape
    square_size = min(h, w)
    x = (w // 2) - (square_size // 2)
    y = (h // 2) - (square_size // 2)
    frame = frame[y:y+square_size, x:x+square_size]
    frame = cv2.resize(frame, target_size)
    # convert BGR to RBG
    frame = frame[:, :, [2, 1, 0]]
    frames.append(frame)
finally:
  cap.release()
sample_video_frames = np.array(frames) / 255.0

print("Sample video shape: %s" % str(sample_video_frames.shape))

Sample video shape: (151, 224, 224, 3)


In [35]:
# load model from tf hub

i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

In [52]:
# predict

# create input tensor
model_input = tf.constant(sample_video_frames, dtype=tf.float32)[tf.newaxis, ...]
logits = i3d(model_input)['default'][0]

# get classification predictions through softmax layer
probabilities = tf.nn.softmax(logits)

print("Top-5 Predictions:")
for i in np.argsort(probabilities)[::-1][:5]:
  print(f"    {labels[i]:22}: {probabilities[i]*100:5.2f}%")

In [ ]:
converted_images = np.clip(sample_video_frames * 255, 0, 255).astype(np.uint8)
imageio.mimsave('./animation.gif', converted_images, fps=35)
embed.embed_file('./animation.gif')

In [ ]:
# !curl -O https://upload.wikimedia.org/wikipedia/commons/8/86/End_of_a_jam.ogv
# video_path = "End_of_a_jam.ogv"